### Transformations And Actions
- sc.parallelize({list}) : list로부터 RDD 만들기

### Actions
- rdd.collect() : RDD 안 내용 보기. 실제 연산에서 사용하면 모든 데이터를 디스크에 가져오기 때문에 지양해야 함.
- rdd.countByValue()
- rdd.take()
- rdd.first()
- rdd.count()
- rdd.foreach(lambda x: f(x)) : foreach한 결과가 바로 나오진 않음. work node에서 결과가 수행될 수 있음. 로그 저장용으로 쓰기도 함.

In [1]:
from pyspark import SparkConf, SparkContext

In [8]:
conf = SparkConf().setMaster("local").setAppName("transformations_actions")
sc = SparkContext(conf = conf)

In [3]:
# spark 연결 정보 확인
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.driver.port', '63823'),
 ('spark.driver.host', '220.79.230.51'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'transformations_actions'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1647010106768'),
 ('spark.app.startTime', '1647010106236'),
 ('spark.ui.showConsoleProgress', 'true')]

In [6]:
# spark 연결 객체 삭제
sc.stop()

In [10]:
foods = sc.parallelize(["짜장면", "마라탕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"])

In [15]:
foods

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [16]:
foods.collect()

['짜장면', '마라탕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [17]:
foods.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '떡볶이': 1,
             '쌀국수': 1,
             '짬뽕': 1,
             '라면': 2,
             '우동': 1})

In [18]:
foods.take(3)

['짜장면', '마라탕', '떡볶이']

In [19]:
foods.take(5)

['짜장면', '마라탕', '떡볶이', '쌀국수', '짬뽕']

In [20]:
foods.first()

'짜장면'

In [21]:
foods.count()

11

In [22]:
foods.distinct().count()

7

In [25]:
foods.foreach(lambda x: print(x))

짜장면
마라탕
떡볶이
쌀국수
짬뽕
짜장면
짜장면
짜장면
라면
우동
라면


### Transformations
- narrow
    - map()
    - flatMap()
    - filter()
    - sample(엘리먼트 재사용 여부, 엘리먼트 사용 비율, 구성 고정)
    - union()
- wide
    - interjection()
    - distinct()
    - reduceByKey()
    - groupByKey()

In [27]:
sc.parallelize([1,2,3]).map(lambda x: x+2).collect()

[3, 4, 5]

In [28]:
movies = [
    '그린 북',
    '매트릭스',
    '토이 스토리',
    '캐스트 어웨이',
    '포드 V 페라리',
    '보헤미안 랩소디',
    '백 투 더 퓨처',
    '반지의 제왕',
    '죽은 시인의 사회'
]

In [29]:
movies_rdd = sc.parallelize(movies)

In [31]:
movies_rdd.collect()

['그린 북',
 '매트릭스',
 '토이 스토리',
 '캐스트 어웨이',
 '포드 V 페라리',
 '보헤미안 랩소디',
 '백 투 더 퓨처',
 '반지의 제왕',
 '죽은 시인의 사회']

In [39]:
map_movies = movies_rdd.map(lambda x: x.split(" "))
map_movies.collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['백', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [40]:
flat_movies = movies_rdd.flatMap(lambda x: x.split(" "))
flat_movies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '백',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [41]:
filtered_movies = flat_movies.filter(lambda x: x != "매트릭스")
filtered_movies.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '백',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [42]:
num1 = sc.parallelize([1,2,3,4])
num2 = sc.parallelize([4,5,6,7,8,9,10])

In [43]:
num1.intersection(num2).collect()

[4]

In [44]:
num1.union(num2).collect()

[1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10]

In [45]:
num1.subtract(num2).collect()

[2, 1, 3]

In [46]:
num_union = num1.union(num2)
num_union.collect()

[1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10]

In [56]:
# num_union.sample(엘리먼트 재사용 여부, 엘리먼트 사용 비율, 구성 고정)
num_union.sample(True, .5, seed=5).collect()

[1, 4, 6, 9, 9, 10]

In [57]:
foods = sc.parallelize(["짜장면", "마라탕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"])

In [59]:
foods_group = foods.groupBy(lambda x: x[0])
foods_group.collect()

[('짜', <pyspark.resultiterable.ResultIterable at 0x7fae4829fb80>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x7fae8889e460>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x7fae8889e2b0>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x7fae8889e640>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x7fae8889ebb0>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x7fae8889e910>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x7fae8889eb20>)]

In [60]:
res = foods_group.collect()

In [61]:
for (k, v) in res:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
떡 ['떡볶이']
쌀 ['쌀국수']
짬 ['짬뽕']
라 ['라면', '라면']
우 ['우동']


In [62]:
nums = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

In [67]:
list(nums.groupBy(lambda x: x % 2).collect()[0][1])

[1, 3, 5, 7, 9]

In [68]:
list(nums.groupBy(lambda x: x % 2).collect()[1][1])

[2, 4, 6, 8, 10]